<a href="https://colab.research.google.com/github/nmills/climate-utility-functions/blob/main/Climate_Analysis_Utility_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploration of Ambient Conditions


In [ ]:
import math
import numpy as np

## Feels Like Temperature

Calculating the apparent "feels like" temperature

Formula adopted from the work done by [Steadman et. al. (1994)](http://www.bom.gov.au/jshess/docs/1994/steadman.pdf) 

In [ ]:
def calculate_water_vapour_pressure(relative_humidity, dry_bulb_temperature):
    """
    Calculate the value for water vapour pressure.
    e = rh / 100 × 6.105 × exp ( 17.27 × Ta / ( 237.7 + Ta ) )

    :parameters
    ----------
    relative_humidity:

    dry_bulb_temperature:

    :return: e or water vapour pressure
    """

    return (relative_humidity / 100) * 6.105 * math.exp((17.27 * dry_bulb_temperature) / (237.7 + dry_bulb_temperature))

In [ ]:
def calculate_apparent_temperature(dry_bulb_temperature, relative_humidity, wind_speed):
    """
    Including the effects of temperature, humidity, and wind:
    AT = Ta + 0.33×e − 0.70×ws − 4.00

    :parameters
    ----------
    dry_bulb_temperature:

    relative_humidity:

    wind_speed:

    :return:
    """

    return (dry_bulb_temperature + 0.33 * calculate_water_vapour_pressure(relative_humidity, dry_bulb_temperature)) \
           - (0.7 * (wind_speed / 3.6)) \
           - 4

In [ ]:
print(calculate_apparent_temperature(10.5, 69.0, 9.0))
print(calculate_apparent_temperature(7.7, 91.0, 9.0))

7.636319392877429
5.101950662573325


The function below will bring our Apparent Temperature calculation functions together in the entry point of a Google Cloud Function. It assumes that the parameters that have been passed are dry_bulb_temperature, relative_humidity, wind_speed.

Once deployed you can use the following line in the cloud console to call the function

`gcloud functions call get-feels-like-temp --data '{ "dry_bulb_temperature":"10.5", "relative_humidity":"69.0", "wind_speed":"9.0" }'

In [ ]:
def cal_app_tmp(request):
    """Responds to any HTTP request.
    Args:
        request (flask.Request): HTTP request object.
    Returns:
        The response text or any set of values that can be turned into a
        Response object using
        `make_response <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>`.
    """
    request_json = request.get_json()

    if not request_json:
        return {"message": "Request parameters corrupted."}, 400

    try:

        dry_bulb_temperature = float(request_json['dry_bulb_temperature'])
        relative_humidity = float(request_json['relative_humidity'])
        wind_speed = float(request_json['wind_speed'])

    except KeyError as err:
        return {"message": "No request parameters available"}, 422

    return str(calculate_apparent_temperature(dry_bulb_temperature, relative_humidity, wind_speed))